In [2]:
from src.models.liquidity_distribution import LiquidityAnalyzer, Pool
from datetime import date
import math
import pandas as pd
from src.data_processing.fetch_data import fetch_oku_liquidity, fetch_data_subgraph

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
POOL_ADDRESS = "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8"
START_DATE = date(2024, 9, 1)
END_DATE = date(2024, 9, 11)

In [10]:
pool_query = """query get_pools($pool_id: ID!) {
  pools(where: {id: $pool_id}) {
    tick
    sqrtPrice
    liquidity
    feeTier
    token0 {
      symbol
      decimals
    }
    token1 {
      symbol
      decimals
    }
  }
}"""
variables = {'pool_id': POOL_ADDRESS}
API_KEY = "893d7471304c5edf436c8ba60781762c"
data_pool = fetch_data_subgraph(API_KEY, pool_query, variables, data_key='pools', first_n=100, batch_size=1000)[0]

Fetched 1 records


In [26]:
# data = fetch_oku_liquidity(pool_address=POOL_ADDRESS, block_number=20719648)

In [28]:
pool = Pool(pool_address=POOL_ADDRESS,
            token0="WETH",
            token1="USDC",
            decimals0=data_pool['token0']['decimals'],
            decimals1=data_pool['token1']['decimals'],
            fee_tier=data_pool['feeTier'],
            sqrt_price_x96=data_pool["sqrtPrice"])

In [6]:
analyzer = LiquidityAnalyzer(
    pool,
    start_date=START_DATE,
    end_date=END_DATE
)
current_tick = int(data["current_pool_tick"])

NameError: name 'data' is not defined

In [1]:
analyzer.sampled_blocks

NameError: name 'analyzer' is not defined

In [18]:
pd.merge(analyzer.sampled_blocks,
         analyzer.liquidity_changes,
         left_on='block_number',
         right_on='block')

,block_number,block_timestamp,cumulative_dollar_volume,datetime,block,tick,active_tick,liquidity,price,amount0,amount1,amount0_real,amount1_real
0,20430379,1722475295,1.024129e+06,2024-08-01 01:21:35,20430379,195527,-887220,1.640134e+15,3.384921e+50,9.038614e+31,2.678000e-07,9.038614e+25,0.000000e+00
1,20430379,1722475295,1.024129e+06,2024-08-01 01:21:35,20430379,195527,-887160,1.715982e+15,3.364674e+50,9.428278e+31,2.811000e-07,9.428278e+25,0.000000e+00
2,20430379,1722475295,1.024129e+06,2024-08-01 01:21:35,20430379,195527,-887100,1.715982e+15,3.344547e+50,9.400037e+31,2.819000e-07,9.400037e+25,0.000000e+00
3,20430379,1722475295,1.024129e+06,2024-08-01 01:21:35,20430379,195527,-887040,1.715982e+15,3.324541e+50,9.371881e+31,2.827000e-07,9.371881e+25,0.000000e+00
4,20430379,1722475295,1.024129e+06,2024-08-01 01:21:35,20430379,195527,-886980,1.715982e+15,3.304654e+50,9.343808e+31,2.836000e-07,9.343808e+25,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12835545,20729203,1726079831,4.641670e+08,2024-09-11 18:37:11,20729203,198693,886980,-1.253468e+08,0.000000e+00,3.965000e-07,1.314333e+32,0.000000e+00,1.314333e+14
12835546,20729203,1726079831,4.641670e+08,2024-09-11 18:37:11,20729203,198693,887040,-1.253468e+08,0.000000e+00,3.953000e-07,1.318282e+32,0.000000e+00,1.318282e+14
12835547,20729203,1726079831,4.641670e+08,2024-09-11 18:37:11,20729203,198693,887100,-1.253468e+08,0.000000e+00,3.942000e-07,1.322242e+32,0.000000e+00,1.322242e+14
12835548,20729203,1726079831,4.641670e+08,2024-09-11 18:37:11,20729203,198693,887160,-1.253468e+08,0.000000e+00,3.930000e-07,1.326215e+32,0.000000e+00,1.326215e+14


In [19]:
df = analyzer.liquidity_changes

In [22]:
df.columns = ['block', 'active_tick', 'tick', 'liquidity', 'price', 'amount0',
       'amount1', 'amount0_real', 'amount1_real']

In [41]:
import numpy as np
df['active_tick_djusted'] = np.floor(df['active_tick'] / pool.tick_spacing) * pool.tick_spacing
df['sb'] = df['tick']-pool.tick_spacing*5
df['sa'] = df['tick']+pool.tick_spacing*5

In [42]:
df.head(2)

,block,active_tick,tick,liquidity,price,amount0,amount1,amount0_real,amount1_real,active_tick_djusted,sb,sa
0,20430379,195527,-887220,1.640134e+15,3.384921e+50,9.038614e+31,2.678000e-07,9.038614e+25,0.0,195480.0,-887520,-886920
1,20430379,195527,-887160,1.715982e+15,3.364674e+50,9.428278e+31,2.811000e-07,9.428278e+25,0.0,195480.0,-887460,-886860


In [43]:
active_df = df[(df['active_tick_djusted'] >= df['sb']) & (df['active_tick_djusted'] <= df['sa'])]

In [48]:
active_df.tail(15)

,block,active_tick,tick,liquidity,price,amount0,amount1,amount0_real,amount1_real,active_tick_djusted,sb,sa
18104,20728120,198911,199020,5.468189e+08,2275.638951,1.408453e+11,6.207858e+19,140845.296202,62.078585,198900.0,198720,199320
18105,20728120,198911,199080,8.444734e+08,2262.026676,1.349929e+11,5.985714e+19,134992.871194,59.857136,198900.0,198780,199380
18106,20728120,198911,199140,1.239980e+09,2248.495825,1.161149e+11,5.179629e+19,116114.886906,51.796288,198900.0,198840,199440
18107,20728120,198911,199200,-1.205872e+09,2235.045913,1.083920e+11,4.864223e+19,108391.969226,48.642230,198900.0,198900,199500
18093,20729203,198693,198360,2.105499e+09,2430.890305,1.639856e+11,6.766173e+19,163985.580815,67.661733,198660.0,198060,198660
18094,20729203,198693,198420,-1.774291e+08,2416.349356,1.638760e+11,6.802340e+19,163875.961490,68.023402,198660.0,198120,198720
18095,20729203,198693,198480,2.472936e+08,2401.895388,1.666370e+11,6.958571e+19,166636.963126,69.585712,198660.0,198180,198780
18096,20729203,198693,198540,3.363394e+07,2387.527880,1.656189e+11,6.957677e+19,165618.900462,69.576770,198660.0,198240,198840
18097,20729203,198693,198600,-6.330146e+08,2373.246314,1.652350e+11,6.983323e+19,165235.035006,69.833232,198660.0,198300,198900
18098,20729203,198693,198660,-8.036493e+07,2359.050177,7.677805e+10,3.731111e+19,76778.053549,37.311106,198660.0,198360,198960


In [50]:
block = df.loc[df['block']==20729203]

In [51]:
block

,block,active_tick,tick,liquidity,price,amount0,amount1,amount0_real,amount1_real,active_tick_djusted,sb,sa
0,20729203,198693,-887220,-3.963300e+08,3.384921e+50,1.444653e+32,4.281000e-07,1.444653e+26,0.000000e+00,198660.0,-887520,-886920
1,20729203,198693,-887160,-1.775695e+09,3.364674e+50,1.481999e+32,4.418000e-07,1.481999e+26,0.000000e+00,198660.0,-887460,-886860
2,20729203,198693,-887100,-1.775695e+09,3.344547e+50,1.477560e+32,4.431000e-07,1.477560e+26,0.000000e+00,198660.0,-887400,-886800
3,20729203,198693,-887040,-1.775695e+09,3.324541e+50,1.473134e+32,4.444000e-07,1.473134e+26,0.000000e+00,198660.0,-887340,-886740
4,20729203,198693,-886980,-1.775695e+09,3.304654e+50,1.468722e+32,4.458000e-07,1.468722e+26,0.000000e+00,198660.0,-887280,-886680
...,...,...,...,...,...,...,...,...,...,...,...,...
29570,20729203,198693,886980,-1.253468e+08,0.000000e+00,3.965000e-07,1.314333e+32,0.000000e+00,1.314333e+14,198660.0,886680,887280
29571,20729203,198693,887040,-1.253468e+08,0.000000e+00,3.953000e-07,1.318282e+32,0.000000e+00,1.318282e+14,198660.0,886740,887340
29572,20729203,198693,887100,-1.253468e+08,0.000000e+00,3.942000e-07,1.322242e+32,0.000000e+00,1.322242e+14,198660.0,886800,887400
29573,20729203,198693,887160,-1.253468e+08,0.000000e+00,3.930000e-07,1.326215e+32,0.000000e+00,1.326215e+14,198660.0,886860,887460


In [56]:
block.loc[block['tick']<block['active_tick'],'amount0_real'] = 0
block.loc[block['tick']>block['active_tick'],'amount1_real'] = 0

In [57]:
block.loc[(block['price']>3200)&(block['price']<3500)]

,block,active_tick,tick,liquidity,price,amount0,amount1,amount0_real,amount1_real,active_tick_djusted,sb,sa
18033,20729203,198693,194760,8.001877e+08,3484.203865,1.284673e+12,3.698212e+20,0.0,369.821195,198660.0,194460,195060
18034,20729203,198693,194820,1.444285e+09,3463.362272,1.270196e+12,3.678541e+20,0.0,367.854142,198660.0,194520,195120
18035,20729203,198693,194880,-6.502686e+08,3442.645347,1.286435e+12,3.747989e+20,0.0,374.798853,198660.0,194580,195180
18036,20729203,198693,194940,1.243972e+09,3422.052345,1.282127e+12,3.757917e+20,0.0,375.791716,198660.0,194640,195240
18037,20729203,198693,195000,-8.999785e+07,3401.582525,1.316290e+12,3.881265e+20,0.0,388.126481,198660.0,194700,195300
18038,20729203,198693,195060,9.852590e+08,3381.235151,1.278661e+12,3.792999e+20,0.0,379.299899,198660.0,194760,195360
18039,20729203,198693,195120,-3.182985e+07,3361.009489,1.275292e+12,3.805773e+20,0.0,380.577278,198660.0,194820,195420
18040,20729203,198693,195180,-1.891584e+09,3340.904811,1.262271e+12,3.789582e+20,0.0,378.958250,198660.0,194880,195480
18041,20729203,198693,195240,-9.643397e+08,3320.920395,1.258694e+12,3.801584e+20,0.0,380.158381,198660.0,194940,195540
18042,20729203,198693,195300,1.967965e+09,3301.055520,1.258491e+12,3.823844e+20,0.0,382.384400,198660.0,195000,195600


In [65]:
block.loc[block['active_tick_djusted']==block['tick']].price.values[0]

np.float64(2359.0501771851)

In [68]:
block['amount_locked'] = block['amount1_real']*block.loc[block['active_tick_djusted']==block['tick']].price.values[0] + block['amount0_real']

/var/folders/11/llb_31fx359c3jzc04q5rklh0000gn/T/ipykernel_90815/101492585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  block['amount_locked'] = block['amount1_real']*block.loc[block['active_tick_djusted']==block['tick']].price.values[0] + block['amount0_real']


In [69]:
block['amount_locked'].sum()

np.float64(60145883.01865217)

In [93]:
current_tick = int(data["current_pool_tick"])
active_tick = (
    math.floor(current_tick / pool.tick_spacing) * pool.tick_spacing
)
sb = active_tick-pool.tick_spacing*10
sa = active_tick+pool.tick_spacing*10



In [99]:
df['amount0_real'].sum()

np.float64(4.9621046174146895e+28)

In [100]:
df['amount1_real'].sum()

np.float64(4.427571698413304e+16)